# 一次性快速训练示例

### 初始化SDK与Anylearn后端引擎连接

In [1]:
from anylearn.config import init_sdk
init_sdk('http://anylearn.xyz', 'username', 'password')

### 获取资源组

#### /!\ 版本0.11.0起，创建训练时，需要额外增加计算资源请求
这里需取到用户想要访问的资源组的ID，后续创建训练时会用到

In [2]:
from anylearn.interfaces import QuotaGroup
groups = QuotaGroup.get_list()
groups

[QuotaGroup(id='QGRP03fe160211ecb6119ef94103bf12', name='METEO', level='Guarantee', node_level='meteo', capacity={'A-100-unique': 4, 'CPU': 32, 'Memory': 128}, default={'A-100-unique': 1, 'CPU': 8, 'Memory': 32}, allocated={'A-100-unique': 1, 'CPU': -3, 'Memory': -4})]

### 创建快速训练任务

#### /!\ 版本0.11.0起，创建训练时，需要额外增加计算资源请求
上一步获取到的资源组的ID在这里会用到

In [3]:
from anylearn.applications.quickstart import quick_train
train_task, algo, dset, project = quick_train(
    algorithm_name="testyhu",
    algorithm_dir="../../resources/cnn",
    dataset_dir="../../resources/fashion_mnist",
    entrypoint="python fashion_mnist.py",
    output="output",
    dataset_hyperparam_name="data-path",
    hyperparams={'batch-size': 256, 'epochs': 3},
    resource_request=[{
        'QGRP03fe160211ecb6119ef94103bf12': {
            'A-100-unique': 1,
            'CPU': 4,
            'Memory': 4,
        }
    }],
)
train_task

Local algorithm (None) has been deleted remotely, forced to re-registering algorithm.


TrainTask(name='yoedbfv7', description='', state=0, visibility=1, creator_id='USERee16e08a11ebbbe9d2ff387e91ee', owner=[], project_id='PROJ4d851bcc4c6586c4caf1f7b060b4', algorithm_id='ALGOb44a1c4411ecb9c1eaf0cae42d3f', train_params='{"data-path": "$DSETe4141c4111ecb9aa424b453a05e6", "batch-size": 256, "epochs": 3}', files='DSETe4141c4111ecb9aa424b453a05e6', results_id='FILE3e461c4411ecb9aa424b453a05e6', secret_key='TKEYd1241c4411ecb9aa424b453a05e6', create_time='2021-09-23 16:03:33', finish_time='', envs='', gpu_num=0, gpu_mem=0, hpo=False, hpo_search_space=None, final_metric=None, resource_request=[{'QGRP03fe160211ecb6119ef94103bf12': {'A-100-unique': 1, 'CPU': 4, 'Memory': 4}}], id='TRAIc6ca1c4411ecb9aa424b453a05e6')

### 追踪训练进度

In [3]:
import time

status = train_task.get_status()
while 'state' not in status:
    print("Waiting...")
    time.sleep(10)
    status = train_task.get_status()
while status['state'] not in ["success", "fail"]:
    if 'process' in status:
        print(f"Progress: {int(100 * float(status['process']))}%")
    else:
        print(status['state'])
    time.sleep(30)
    status = train_task.get_status()
    print(status)
status['state']

working
{'ip': '10.0.1.157', 'secret_key': 'TKEY38a4cffd11eb8932f292118be07c', 'state': 'success'}


'success'